In [1]:
%cd ..

e:\Github\plant_disease_detection


# Importing Libraries

In [2]:
import os
import pandas as pd
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Assigning to use GPU instead of CPU

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Image Transformations

In [4]:
img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(
        brightness=0.2, 
        contrast=0.2, 
        saturation=0.2, 
        hue=0.1
    ),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
        )
])

In [5]:
train_ds = datasets.ImageFolder(root = r'dataset\datasets\vipoooool\new-plant-diseases-dataset\versions\2\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train', transform=img_transforms)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)

In [6]:
val_ds = datasets.ImageFolder(root = r'dataset\datasets\vipoooool\new-plant-diseases-dataset\versions\2\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid', transform=img_transforms)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# Intializing the model we will be using `Resnet18`

In [7]:
model = models.resnet18(pretrained=True)

e:\Anaconda\envs\plant_disease\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Anaconda\envs\plant_disease\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
len(train_ds.classes)

38

In [10]:
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_features=512, out_features=len(train_ds.classes), bias=True)
)

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
model = model.to(device)

# Assiging the loss function & Optimizer

In [12]:
loss_func = nn.CrossEntropyLoss()

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Assigning Variables for dynamic 

In [14]:
epochs = 8

# Training our model

In [15]:
for epoch in range(epochs):
    model.train()

    total_loss = 0.0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, total=len(train_loader)):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outs = model(imgs)
        loss_val = loss_func(outs, labels)
        loss_val.backward()
        optimizer.step()

        total_loss += loss_val.item()


        _, predicted = torch.max(outs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


100%|██████████| 1099/1099 [08:04<00:00,  2.27it/s]


Epoch 1/8, Loss: 0.2404, Accuracy: 93.14%


100%|██████████| 1099/1099 [07:05<00:00,  2.59it/s]


Epoch 2/8, Loss: 0.0941, Accuracy: 97.00%


100%|██████████| 1099/1099 [06:21<00:00,  2.88it/s]


...

Epoch 6/8, Loss: 0.0490, Accuracy: 98.48%


100%|██████████| 1099/1099 [06:42<00:00,  2.73it/s]


Epoch 7/8, Loss: 0.0429, Accuracy: 98.66%


100%|██████████| 1099/1099 [07:13<00:00,  2.53it/s]

Epoch 8/8, Loss: 0.0405, Accuracy: 98.76%


# validating our model performance

In [16]:
model.eval() 

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outs = model(imgs)
        _, predicted = torch.max(outs, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        

accuracy = 100 * correct / total


print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 98.93%


# Exporting the model in an onnx format

In [17]:
dummy_input = torch.randn(1, 3, 256, 256).to(device)

torch.onnx.export(
    model,                       
    dummy_input,                 
    "plant_disease_model.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)